In [ ]:
%cd /content/drive/MyDrive/MLOM_Lab6

/content/drive/MyDrive/MLOM_Lab6


In [ ]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential # used o give an order 
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Load ascii text and convert to lowercase

In [ ]:
filename = "data.txt"
raw_text = open(filename, 'r', encoding = 'utf-8').read()
raw_text = raw_text.lower()

In [ ]:
int_to_char = dict((i,c) for i,c in enumerate(chars))

In [ ]:
#To do.change the value of seq_length and check the training set.Can also add tests dataset
raw_text[0:100]

"project gutenberg's alice's adventures in wonderland, by lewis carroll\n\nthis ebook is for the use of"

In [ ]:
#clean dataset to remove unwanted characters
#create mapping of unique chars to integers\n"
chars = sorted(list(set(raw_text)))
print(chars)
char_to_int = dict((c,i) for i, c in enumerate(chars))

['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
print(char_to_int)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, "'": 7, '(': 8, ')': 9, '*': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '?': 27, '@': 28, '[': 29, ']': 30, '_': 31, 'a': 32, 'b': 33, 'c': 34, 'd': 35, 'e': 36, 'f': 37, 'g': 38, 'h': 39, 'i': 40, 'j': 41, 'k': 42, 'l': 43, 'm': 44, 'n': 45, 'o': 46, 'p': 47, 'q': 48, 'r': 49, 's': 50, 't': 51, 'u': 52, 'v': 53, 'w': 54, 'x': 55, 'y': 56, 'z': 57}


In [ ]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total characters: ",n_chars)
print("Total Vocab(Unique Characters): ",n_vocab)

Total characters:  163780
Total Vocab(Unique Characters):  58


In [ ]:
#prepare the dataset of input to output pairs encoded as integers 
seq_length = 15 #can be changed
dataX = []
dataY = []
for i in range(0,n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataY)
print("Total Patterns: ", n_patterns)

Total Patterns:  163765


In [ ]:
#Checking dataX and dataY
print(dataX[1])
print(dataY[1])

[49, 46, 41, 36, 34, 51, 1, 38, 52, 51, 36, 45, 33, 36, 49]
38


In [ ]:
#reshape X to be [samples, time seteps, features]
X = np.reshape(dataX,(n_patterns,seq_length,1))

#normalize - rescaling the integer values
X = X / float(n_vocab)
print(X)


[[[0.81034483]
  [0.84482759]
  [0.79310345]
  ...
  [0.77586207]
  [0.56896552]
  [0.62068966]]

 [[0.84482759]
  [0.79310345]
  [0.70689655]
  ...
  [0.56896552]
  [0.62068966]
  [0.84482759]]

 [[0.79310345]
  [0.70689655]
  [0.62068966]
  ...
  [0.62068966]
  [0.84482759]
  [0.65517241]]

 ...

 [[0.55172414]
  [0.56896552]
  [0.79310345]
  ...
  [0.79310345]
  [0.79310345]
  [0.72413793]]

 [[0.56896552]
  [0.79310345]
  [0.89655172]
  ...
  [0.79310345]
  [0.72413793]
  [0.86206897]]

 [[0.79310345]
  [0.89655172]
  [0.87931034]
  ...
  [0.72413793]
  [0.86206897]
  [0.22413793]]]


In [ ]:
#one hot encode the output variable
y = np_utils.to_categorical(dataY)
#print(dataY)

In [ ]:
#Define the LSTM Model
model = Sequential()
model.add(LSTM(256,input_shape=(X.shape[1],X.shape[2])))#It can have 1 or more training samples
model.add(Dropout(0.2))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer = 'adam')

#define the checkpoint
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor = 'loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
#change the hyperparameter values and train the model
epochs = 10
batch_size = 128
model.fit(X,y, epochs = epochs, batch_size = batch_size, callbacks = callbacks_list)

NameError: ignored

In [ ]:
#load the network weights
filename = "weights-improvement-10-2.3100.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy',optimizer='adam')

OSError: ignored

In [ ]:
#generate a random seed
print(len(dataX))
start = np.random.randint(0,len(dataX)-1)
print(start)
pattern = dataX[start] #dataX contains list of patterns
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

#generate characters
length = 10
final = []
for i in range(length):
  #reshaping the seed sequnce before passing it to the LSTM model
  x = np.reshape(pattern,(1,len(pattern),1))
  #print(x)
  #normalizing the integer values
  x = x / float(n_vocab)

  #print(x)
  #making prediction
  prediction = model.predict(x,verbose=0)

  #Get the predicted value with the maximum probability
  index = np.argmax(prediction)

  #Convert the predicted integer to char
  result = int_to_char[index]
  #print(result)
  final.append(result)

  #Adding the predicted character to the sequence
  pattern.append(index)

  #Removing the first character from the seed seqeunce
  pattern = pattern[1:len(pattern)]
print(final)


163765
139744
Seed:
" nt on, spreadin "
['6', '6', '6', '6', '6', '6', '6', '6', '6', '6']
